In [1]:
A = lambda l,a4=1,a3=1,a2=1: (0.6/a4+0.25/a3*9+0.14/a2*(l-1)*10)/(l*l*l*10)

In [2]:
A(25,1,9,240)*25*25*25*10

0.99

In [3]:
(1/A(25),1/A(50),1/A(75))

(4286.694101508916, 17494.75157452764, 39631.28229215594)

In [4]:
def a4(zl:list[tuple[int]],z0:tuple[int]):
    counter = 1
    for z in zl:
        if z == z0:
            counter += 1
    return counter

zl1 = [(25,25,25,10)]
assert a4(zl1,(25,25,25,10)) == 2

def a3(zl:list[tuple[int]],z0:tuple[int]):
    counter = 1
    for z in zl:
        if z[:3] == z0[:3] and not z == z0:
            counter += 1
    return counter

zl2 = [(25,25,25,9),(25,25,25,10)]
assert a3(zl2,(25,25,25,1)) == 3
assert a3(zl2,(25,25,25,10)) == 2

zla25 = [(z1,z2,z3,z4) for z1 in range(1,25+1) for z2 in range(1,25+1) for z3 in range(1,25+1) for z4 in range(1,11)]

def a2(zl:list[tuple[int]],z0:tuple[int]):
    counter = 1
    for z in zl:
        if z[:2] == z0[:2] and not z[:3] == z0[:3]:
            counter += 1
    return counter

def eval_z(zl:list[tuple[int]],z0: tuple[int],l=25):
    return A(l,a4(zl,z0),a3(zl,z0),a2(zl,z0))

def eval_l(zl:list[tuple[int]],mz:list[tuple[int]],l=25):
    (z2,z3,z4) = z2z3z4(zl,l)
    (mz2,mz3,mz4) = z2z3z4(mz,l)
    return (sum([0.6/(z4[z]+mz4[z])*mz4[z] for z in mz4])+sum([0.25/(z3[z]+1)*(mz3[z]*(mz3[z]-1)+(10-mz3[z])*mz3[z]) for z in mz3])+sum([0.14/(z2[z]+1)*(mz2[z]*(mz2[z]-10)+(l*10-mz2[z])*mz2[z]) for z in mz2]))/(l*l*l*10)

def z2z3z4(zl:list[tuple[int]],l=25):
    z2 = {(z1,z2):0 for z1 in range(1,l+1) for z2 in range(1,l+1)}
    z3 = {(z1,z2,z3):0 for z1 in range(1,l+1) for z2 in range(1,l+1) for z3 in range(1,l+1)}
    z4 = {(z1,z2,z3,z4):0 for z1 in range(1,l+1) for z2 in range(1,l+1) for z3 in range(1,l+1) for z4 in range(1,11)}
    for z in zl:
        z2[z[:2]] += 1
        z3[z[:3]] += 1
        z4[z] += 1
    return (z2,z3,z4)

def min_elem(zk:dict[tuple]):
    n = 0
    while True:
        for z in zk:
            if zk[z] == n:
                return z
        n += 1

import sys
def best1(zl:list[tuple[int]], l=25 ) -> tuple[tuple,float]:
    (z2,z3,z4) = z2z3z4(zl,l)
    # minz2 = min_elem(z2)
    # minz3 = min_elem({z:z3[z] for z in z3 if z[:2] == minz2 })
    # minz4 = min_elem({z:z4[z] for z in z4 if z[:3] == minz3 })
    # return minz4
    mx = 0.0
    zmx = None
    for z in [(z1,z2,z3,z4) for z1 in range(1,l+1) for z2 in range(1,l+1) for z3 in range(1,l+1) for z4 in range(1,11)]:
        v = A(l,z4[z]+1,z3[z[:3]]+1,z2[z[:2]]+1)
        if v > mx:
            zmx = z
            mx = v
    return zmx,mx

In [5]:
(z,a) = best1(zla25); (z,1/a)

((1, 1, 1, 1), 244748.67387172728)

In [6]:
(z,a) = best1(zl2+[(1,1,1,1)]); (z,1/a)

((1, 2, 1, 1), 4286.694101508916)

In [95]:
zlo = {}
games = {}
thjson = {}
import json
for n in range(13,23):
    games[n] = json.load(open(f"lotteries/game-info-{n}.json"))
    thjson[n] = json.load(open(f"lotteries/ticket-history-{n}.json"))
    zlo[n] = [(int(e["numbers"][0]),int(e["numbers"][1]),int(e["numbers"][2]),int(e["etherball"])) for e in thjson[n]]
(z,a) = best1(zlo[20]); print(z,1/a)
(z,a) = best1(zlo[17]); print(z,1/a)
(z,a) = best1(zlo[13]); print(z,1/a)
print(25*25,25*25*25,25*25*25*10)

(1, 6, 1, 1) 4286.694101508916
(1, 1, 1, 1) 11120.996441281139
(1, 1, 1, 1) 7951.653944020355
625 15625 156250


In [107]:
def player_ticket_count(thj,n=10):
    res = {}
    for e in thj:
        if e["player"] in res:
            res[e["player"]] += 1
        else:
            res[e["player"]] = 1
    return sorted(res.items(),reverse=True,key= lambda e: e[1])[:n]

def winners(n):
    wns = games[n]["winningNumbers"]
    winningNumbers = (int(wns[0]),int(wns[1]),int(wns[2]),int(wns[3]))
    winner_counts = [0,0,int(games[n]["bronzeWinners"]),int(games[n]["silverWinners"]),int(games[n]["goldWinners"])]
    price_pool = int(games[n]["prizePool"]) #/1000000000000000000
    print(winningNumbers)
    print("payouts ",[int(p) for p in games[n]["payouts"]])
    print("expected",[
          int(6*price_pool/(10*winner_counts[4])) if winner_counts[4] > 0 else 0,
          int(price_pool/(4*winner_counts[3])) if winner_counts[3]>0 else 0,
          int(float(0.14/winner_counts[2])*price_pool) if winner_counts[2]>0 else 0
          ])
    w4 = 0
    w3 = 0
    w2 = 0
    for k in range(len(thjson[n])):
        if winningNumbers == zlo[n][k]:
            print(zlo[n][k],4,thjson[n][k]["player"])
            w4+=1
        elif winningNumbers[:3] == zlo[n][k][:3]:
            print(zlo[n][k],3,thjson[n][k]["player"])
            w3 += 1
        elif winningNumbers[:2] == zlo[n][k][:2]:
            print(zlo[n][k],2,thjson[n][k]["player"])
            w2 += 1
    assert winner_counts[4] == w4
    assert winner_counts[3] == w3+w4
    assert winner_counts[2] == w2+w3+w4


In [96]:
for n in zlo:
    print(
        n,
        len(zlo[n]),
        len([ z for z,v in z2z3z4(zlo[n])[0].items() if v == 0 ]),
        int(games[n]["prizePool"])/1000000000000000000,
        [int(games[n]["goldWinners"]),int(games[n]["silverWinners"]),int(games[n]["bronzeWinners"])] if games[n]["status"] == 2 else "ongoing",
        player_ticket_count(thjson[n],3)                                                      
        )

13 1072 0 8209.061790865204 [0, 0, 1] [('0xbaCaA88896c612506d131F5FA907D57a45C905b2', 144), ('0x85F321EAf349264C264dFD01599E56841B653Ef3', 19), ('0x4dd477166BF6755062cd8783486AC440BA08d5AB', 18)]
14 969 0 7946.702522235423 [0, 0, 1] [('0xbaCaA88896c612506d131F5FA907D57a45C905b2', 186), ('0x7E2D995684CB27e4E687573F5BA02CA646AEeAb7', 30), ('0x3b2545f361B2fcA136E86667864451DbD1583cc1', 21)]
15 839 167 7593.697143900109 [0, 0, 2] [('0x4f6fb157d32D0E2Bdfb1D3236Cf1e3F86c9828FB', 20), ('0x91B6Bff57D2E903014D306Cb35d9028b3b52f49E', 20), ('0xbb4A0d90c517A2F76d9A9B720132F62062146de1', 12)]
16 731 194 7185.642572315093 [0, 0, 0] [('0xe740d6E029569fbEb6585682c3ab91304cC69C12', 35), ('0x95EdEEbeEe2D4417369dB9E5448EF26C15f3641E', 33), ('0xb85e5769cC4A62d7DEfC3113aEe8E06Cd02a96C6', 18)]
17 1737 0 8850.786146591941 [0, 0, 2] [('0x25B2884de732812228500855c214207108A166D1', 625), ('0xbaCaA88896c612506d131F5FA907D57a45C905b2', 152), ('0x4f6fb157d32D0E2Bdfb1D3236Cf1e3F86c9828FB', 55)]
18 1577 0 9100.16822

In [108]:
zlo_max = max(zlo.keys())
for n in zlo:
    if n == zlo_max: continue
    print(n)
    winners(n)

13
(23, 17, 17, 2)
payouts  [0, 0, 1149268650721128476789]
expected [0, 0, 1149268650721128611840]
(23, 17, 23, 10) 2 0x384849023279C69018E9C5Bdbcb2127CE4DBd109
14
(19, 18, 23, 1)
payouts  [0, 0, 1112538353112959205271]
expected [0, 0, 1112538353112959418368]
(19, 18, 17, 8) 2 0xCC5c24dDF78B2626A25b4AFC6a305371D39ae757
15
(11, 14, 9, 2)
payouts  [0, 0, 531558800073007662240]
expected [0, 0, 531558800073007693824]
(11, 14, 4, 5) 2 0x582247d7E2Ab82cBae8788Ea896EA81F7939C4CF
(11, 14, 24, 6) 2 0x4f6fb157d32D0E2Bdfb1D3236Cf1e3F86c9828FB
16
(14, 6, 12, 1)
payouts  [0, 0, 0]
expected [0, 0, 0]
17
(13, 12, 5, 10)
payouts  [0, 0, 619555030261435947775]
expected [0, 0, 619555030261435990016]
(13, 12, 2, 2) 2 0x9ef4cb0C558985c53139cD47F17374A1C90f6450
(13, 12, 2, 9) 2 0x25B2884de732812228500855c214207108A166D1
18
(14, 13, 15, 4)
payouts  [5460100934761890476652, 2275042056150787698605, 254804710288888222243]
expected [5460100934761890971648, 2275042056150787817472, 254804710288888266752]
(14, 13,

In [10]:

zl = zlo[20].copy()
def my_z(zl,l=25):
    mz = []
    for n in range(1,10):
        z,_ = best1(zl)
        zl.append(z)
        mz.append(z)
        J = len(zl)
        (z2,z3,z4) = z2z3z4(zl,l)
        print(mz,zl)
        print(z4[z],z3[z[:3]]-z4[z],z2[z[:2]]-z3[z[:3]])
        print(z,J,n/sum([A(l,z4[z],z3[z[:3]]-z4[z],z2[z[:2]]-z3[z[:3]]) for z in mz]) - n)

my_z(zl)


[(1, 6, 1, 1)] [(2, 20, 16, 1), (23, 6, 6, 5), (14, 20, 13, 4), (20, 24, 4, 2), (23, 6, 23, 6), (17, 23, 9, 7), (19, 7, 9, 1), (3, 4, 21, 5), (1, 2, 24, 6), (15, 20, 1, 2), (10, 11, 12, 1), (2, 7, 23, 2), (23, 9, 7, 10), (12, 17, 10, 3), (13, 18, 23, 10), (11, 5, 4, 6), (2, 1, 25, 4), (24, 21, 18, 7), (20, 19, 19, 4), (12, 20, 3, 5), (13, 12, 18, 9), (22, 25, 11, 10), (13, 18, 19, 2), (16, 2, 15, 1), (8, 2, 6, 4), (12, 12, 23, 8), (3, 21, 7, 2), (10, 10, 10, 5), (1, 10, 11, 9), (20, 18, 19, 3), (13, 8, 22, 5), (16, 22, 20, 9), (14, 17, 9, 7), (15, 11, 6, 9), (10, 17, 9, 8), (21, 18, 6, 5), (21, 22, 11, 10), (2, 4, 14, 6), (15, 8, 11, 7), (20, 15, 24, 6), (7, 4, 2, 7), (4, 23, 15, 5), (20, 22, 6, 8), (24, 23, 5, 1), (1, 1, 11, 4), (12, 4, 24, 5), (18, 12, 16, 3), (1, 9, 8, 5), (25, 16, 20, 7), (15, 22, 15, 9), (5, 25, 3, 9), (9, 6, 17, 6), (21, 24, 17, 5), (19, 14, 3, 10), (21, 4, 6, 9), (20, 16, 13, 10), (2, 2, 4, 6), (16, 21, 9, 4), (23, 23, 22, 3), (24, 25, 11, 6), (17, 8, 19, 7), (2

ZeroDivisionError: float division by zero

In [ ]:
def test1():
    l = 25
    my_z = zla25.copy()
    
    zl = zla25.copy()
    return eval_l([],my_z)
test1()

36.45000000000103

In [ ]:
with open("lotteries/20.py") as fd:
    zl20 = eval(fd.read())

In [ ]:
zl20

[(2, 20, 16, 1),
 (23, 6, 6, 5),
 (14, 20, 13, 4),
 (20, 24, 4, 2),
 (23, 6, 23, 6),
 (17, 23, 9, 7),
 (19, 7, 9, 1),
 (3, 4, 21, 5),
 (1, 2, 24, 6),
 (15, 20, 1, 2),
 (10, 11, 12, 1),
 (2, 7, 23, 2),
 (23, 9, 7, 10),
 (12, 17, 10, 3),
 (13, 18, 23, 10),
 (11, 5, 4, 6),
 (2, 1, 25, 4),
 (24, 21, 18, 7),
 (20, 19, 19, 4),
 (12, 20, 3, 5),
 (13, 12, 18, 9),
 (22, 25, 11, 10),
 (13, 18, 19, 2),
 (16, 2, 15, 1),
 (8, 2, 6, 4),
 (12, 12, 23, 8),
 (3, 21, 7, 2),
 (10, 10, 10, 5),
 (1, 10, 11, 9),
 (20, 18, 19, 3),
 (13, 8, 22, 5),
 (16, 22, 20, 9),
 (14, 17, 9, 7),
 (15, 11, 6, 9),
 (10, 17, 9, 8),
 (21, 18, 6, 5),
 (21, 22, 11, 10),
 (2, 4, 14, 6),
 (15, 8, 11, 7),
 (20, 15, 24, 6),
 (7, 4, 2, 7),
 (4, 23, 15, 5),
 (20, 22, 6, 8),
 (24, 23, 5, 1),
 (1, 1, 11, 4),
 (12, 4, 24, 5),
 (18, 12, 16, 3),
 (1, 9, 8, 5),
 (25, 16, 20, 7),
 (15, 22, 15, 9),
 (5, 25, 3, 9),
 (9, 6, 17, 6),
 (21, 24, 17, 5),
 (19, 14, 3, 10),
 (21, 4, 6, 9),
 (20, 16, 13, 10),
 (2, 2, 4, 6),
 (16, 21, 9, 4),
 (23, 23, 